In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)
np.random.shuffle(X_train.to_numpy())
train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255

m = X_train.shape[0] # Number of Training Samples

X_valid = X_train.iloc[(int(0.85*m)):]
valid_class = train_class[(int(0.85*m)):]
X_train = X_train.iloc[0:int(0.85*m)]
train_class = train_class[0:int(0.85*m)]


m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The total number of validation samples = {}".format(X_valid.shape[0]))

print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 11050
The total number of validation samples = 1950
The number of features = 784


In [3]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
valid_class_enc = pd.get_dummies(valid_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [4]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_valid = np.hstack((np.ones((X_valid.shape[0],1)), X_valid.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [31]:
lr = 0.1
arch_test = [1,5,10,50,100]
arch = [arch_test[3]] #means one hidden layer with 2 perceptrons 
batch_size = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [6]:
#Mini-Batch formation
mini_batch = [(X_train[i:i+batch_size,:], train_class_enc[i:i+batch_size]) for i in range(0, m, batch_size)]
print("The number of mini-batches formed is = {}".format(len(mini_batch)))

The number of mini-batches formed is = 111


In [7]:
#Theta Initialization 
def theta_init(arch=[50]):
    theta = []
    for i in range(len(arch)+1):
        if i == 0:
            dim0=n+1
            dim1=arch[i]
        elif (i == len(arch)):
            dim0=arch[i-1]
            dim1 = r
        else:
            dim0=arch[i-1]
            dim1= arch[i]

        theta.append(2*np.random.random((dim0, dim1))-1)
        #theta.append(np.zeros((dim0, dim1)))
    return theta

In [8]:
def activation(x):
    return 1/(1+np.exp(-x))

In [9]:
def forward_prop(data, theta):
    fm = []
    fm.append(data)
    for l in range(len(theta)):
        fm.append(activation(np.dot(fm[l], theta[l])))
    return fm

In [10]:
def cost_total(X, theta, Y, m):
    fm = forward_prop(X, theta)
    cost = (1/(2*m))*np.sum((Y-fm[-1])**2)
    return cost

In [11]:
def calc_accuracy(data, theta, actual_class):
    pred_class = forward_prop(data, theta)
    test_pred_class = pred_class[-1]
    for i in range(len(test_pred_class)):
        test_pred_class[i][test_pred_class[i] == np.max(test_pred_class[i])] = 1
        test_pred_class[i][test_pred_class[i] != np.max(test_pred_class[i])] = 0


    test_acc = 0
    for i in range(len(actual_class)):
        if (np.array_equal(test_pred_class[i], actual_class[i])):
            test_acc+=1
    test_acc /= data.shape[0]

    #print("The Test Accuracy of the model = {}%".format(test_acc*100))
    return (test_acc*100)

In [12]:
epochs = []
train_accuracy = []
test_accuracy = []
valid_accuracy=[]
train_time = []

In [36]:
arch_test = [1, 5, 10, 50, 100]
#arch_test = [50]

In [ ]:
lr0=1.5
for i in range(len(arch_test)):
    theta = theta_init([arch_test[i]])
    fm = forward_prop(X_train, theta)
    epoch = 1
    start = time.time()
    cost_init = cost_total(X_valid, theta, valid_class_enc, m)

    while(True):
        count = 0
        lr = lr0/(np.power(epoch, 1/3))
        print("learning rate = ", lr)

        print("Initial Cost on Val dataset for this epoch {} = {}".format(epoch, cost_init))

        for b in mini_batch:
            X_b = b[0]
            Y_b = b[1]
            fm = forward_prop(X_b, theta)
            delta = [None]*len(fm)

            if (count % 60 == 0):
                print("Error on this batch = "+str(cost_total(X_b, theta, Y_b, batch_size)))
            #Backward Propagation

            for l in range(len(fm)-1, 0, -1):
                if (l == len(fm)-1):
                    delta[l] = ((1/batch_size)*(Y_b - fm[l])*fm[l]*(1-fm[l]))
                else:
                    delta[l]=(np.dot(delta[l+1], theta[l].T)*fm[l]*(1-fm[l]))

            for t in range(len(theta)):
                theta[t] += lr*np.dot(fm[t].T, delta[t+1]) 

            count+=1
        epoch+=1 #Number of epochs
        
        cost_final = cost_total(X_valid, theta, valid_class_enc, m)
        print("Cost on val dataset after {} epochs is = {}".format(epoch, cost_final))
        if (abs(cost_final-cost_init) < 1e-07):
            print("cost initial= {} , cost final={} , change in cost= {}".format(cost_init,cost_final, cost_final-cost_init))
            break
        cost_init = cost_final
    epochs.append(epoch)
    train_time.append(time.time()-start)
    train_accuracy.append(calc_accuracy(X_train, theta, train_class_enc))
    valid_accuracy.append(calc_accuracy(X_valid, theta, valid_class_enc))
    test_accuracy.append(calc_accuracy(X_test, theta, test_actual_class_enc))
    print("\n------------------------------------------------------------------------------")
    print("The stats for number of units in the hidden layer = {} are as below:".format(arch_test[i]))
    print("------------------------------------------------------------------------------")
    print("The number of epochs = {:2.3f}".format(epochs[-1]))
    print("The training time = {:2.3f}sec".format(train_time[-1]))
    print("The training accuracy is = {:2.3f}%".format(train_accuracy[-1]))
    print("The validation accuracy is = {:2.3f}%".format(valid_accuracy[-1]))
    print("The test accuracy is = {:2.3f}%".format(test_accuracy[-1]))
    print("------------------------------------------------------------------------------\n")

learning rate =  1.5
Initial Cost on Val dataset for this epoch 1 = 0.6237366296578414
Error on this batch = 3.5088345346628147
Error on this batch = 3.2503330591730224
Cost on val dataset after 2 epochs is = 0.5735459576902762
learning rate =  1.1905507889761495
Initial Cost on Val dataset for this epoch 2 = 0.5735459576902762
Error on this batch = 3.2500419652773984
Error on this batch = 3.2498406027875912
Cost on val dataset after 3 epochs is = 0.573504310178997
learning rate =  1.040041911525952
Initial Cost on Val dataset for this epoch 3 = 0.573504310178997
Error on this batch = 3.2498871115243926
Error on this batch = 3.249546208378276
Cost on val dataset after 4 epochs is = 0.5734459308373425
learning rate =  0.944940787421155
Initial Cost on Val dataset for this epoch 4 = 0.5734459308373425
Error on this batch = 3.249598229216718
Error on this batch = 3.248327632236004
Cost on val dataset after 5 epochs is = 0.5728770117371392
learning rate =  0.8772053214638599
Initial Cost o

Error on this batch = 0.480727380926359
Cost on val dataset after 37 epochs is = 0.0849720376439701
learning rate =  0.4501501000778427
Initial Cost on Val dataset for this epoch 37 = 0.0849720376439701
Error on this batch = 0.4807739592346358
Error on this batch = 0.480725207429481
Cost on val dataset after 38 epochs is = 0.08497124967443612
learning rate =  0.4461662619442522
Initial Cost on Val dataset for this epoch 38 = 0.08497124967443612
Error on this batch = 0.48077217537710154
Error on this batch = 0.48072331400266993
Cost on val dataset after 39 epochs is = 0.08497052221486104
learning rate =  0.44231980966270107
Initial Cost on Val dataset for this epoch 39 = 0.08497052221486104
Error on this batch = 0.48077063762927774
Error on this batch = 0.48072165882981077
Cost on val dataset after 40 epochs is = 0.08496984729791038
learning rate =  0.43860266073192994
Initial Cost on Val dataset for this epoch 40 = 0.08496984729791038
Error on this batch = 0.48076930834445064
Error on 

Cost on val dataset after 72 epochs is = 0.08495845522580857
learning rate =  0.3605623925768521
Initial Cost on Val dataset for this epoch 72 = 0.08495845522580857
Error on this batch = 0.4807595497861219
Error on this batch = 0.48070779847637907
Cost on val dataset after 73 epochs is = 0.08495822205634493
learning rate =  0.3589084133919559
Initial Cost on Val dataset for this epoch 73 = 0.08495822205634493
Error on this batch = 0.4807595013763009
Error on this batch = 0.48070771566625553
Cost on val dataset after 74 epochs is = 0.08495799219466522
learning rate =  0.3572843712035789
Initial Cost on Val dataset for this epoch 74 = 0.08495799219466522
Error on this batch = 0.4807594555936352
Error on this batch = 0.48070763740568584
Cost on val dataset after 75 epochs is = 0.0849577655139562
learning rate =  0.35568933044900636
Initial Cost on Val dataset for this epoch 75 = 0.0849577655139562
Error on this batch = 0.4807594122023919
Error on this batch = 0.4807075633714193
Cost on va

Error on this batch = 0.4807063936500477
Cost on val dataset after 108 epochs is = 0.08495156504135033
learning rate =  0.3149802624737183
Initial Cost on Val dataset for this epoch 108 = 0.08495156504135033
Error on this batch = 0.4807586110734775
Error on this batch = 0.4807063771616106
Cost on val dataset after 109 epochs is = 0.08495140621493891
learning rate =  0.31401405927925385
Initial Cost on Val dataset for this epoch 109 = 0.08495140621493891
Error on this batch = 0.4807585961567692
Error on this batch = 0.4807063613110944
Cost on val dataset after 110 epochs is = 0.08495124871595708
learning rate =  0.31305960336348476
Initial Cost on Val dataset for this epoch 110 = 0.08495124871595708
Error on this batch = 0.48075858153104395
Error on this batch = 0.4807063460701962
Cost on val dataset after 111 epochs is = 0.0849510925213516
learning rate =  0.3121166470390388
Initial Cost on Val dataset for this epoch 111 = 0.0849510925213516
Error on this batch = 0.4807585671858899
Err

Cost on val dataset after 143 epochs is = 0.08494667280194265
learning rate =  0.2868440945715393
Initial Cost on Val dataset for this epoch 143 = 0.08494667280194265
Error on this batch = 0.4807582142053461
Error on this batch = 0.48070606073416616
Cost on val dataset after 144 epochs is = 0.08494654997913083
learning rate =  0.28617856063833297
Initial Cost on Val dataset for this epoch 144 = 0.08494654997913083
Error on this batch = 0.4807582055805152
Error on this batch = 0.48070605616842105
Cost on val dataset after 145 epochs is = 0.08494642794525561
learning rate =  0.2855191606965982
Initial Cost on Val dataset for this epoch 145 = 0.08494642794525561
Error on this batch = 0.4807581970626567
Error on this batch = 0.48070605174528797
Cost on val dataset after 146 epochs is = 0.08494630669028105
learning rate =  0.28486579648931415
Initial Cost on Val dataset for this epoch 146 = 0.08494630669028105
Error on this batch = 0.4807581886493159
Error on this batch = 0.4807060474593635

...

Error on this batch = 0.4353661483058022
Cost on val dataset after 1331 epochs is = 0.07513671991486728
learning rate =  0.13636363636363638
Initial Cost on Val dataset for this epoch 1331 = 0.07513671991486728
Error on this batch = 0.4353950970361591
Error on this batch = 0.4353626485752078
Cost on val dataset after 1332 epochs is = 0.0751362210244963
learning rate =  0.13632950278614941
Initial Cost on Val dataset for this epoch 1332 = 0.0751362210244963
Error on this batch = 0.4353909633815956
Error on this batch = 0.4353591530987528
Cost on val dataset after 1333 epochs is = 0.07513572357296898
learning rate =  0.13629540335931778
Initial Cost on Val dataset for this epoch 1333 = 0.07513572357296898
Error on this batch = 0.43538682976761084
Error on this batch = 0.4353556616791785
Cost on val dataset after 1334 epochs is = 0.07513522755362044
learning rate =  0.13626133802339271
Initial Cost on Val dataset for this epoch 1334 = 0.07513522755362044
Error on this batch = 0.4353826956

Error on this batch = 0.435248455224026
Cost on val dataset after 1365 epochs is = 0.07512047540960377
learning rate =  0.13522190249857688
Initial Cost on Val dataset for this epoch 1365 = 0.07512047540960377
Error on this batch = 0.4352519334133589
Error on this batch = 0.43524500197465693
Cost on val dataset after 1366 epochs is = 0.07512001566348972
learning rate =  0.13518889739536236
Initial Cost on Val dataset for this epoch 1366 = 0.07512001566348972
Error on this batch = 0.43524758130108027
Error on this batch = 0.4352415476161404
Cost on val dataset after 1367 epochs is = 0.07511955664089204
learning rate =  0.1351559244922508
Initial Cost on Val dataset for this epoch 1367 = 0.07511955664089204
Error on this batch = 0.4352432195230664
Error on this batch = 0.43523809206137853
Cost on val dataset after 1368 epochs is = 0.07511909831765538
learning rate =  0.13512298373430656
Initial Cost on Val dataset for this epoch 1368 = 0.07511909831765538
Error on this batch = 0.43523884

Cost on val dataset after 1399 epochs is = 0.07510514542337861
learning rate =  0.1341174706016305
Initial Cost on Val dataset for this epoch 1399 = 0.07510514542337861
Error on this batch = 0.4350984364408492
Error on this batch = 0.43512666912552944
Cost on val dataset after 1400 epochs is = 0.07510470194179018
learning rate =  0.13408553026448516
Initial Cost on Val dataset for this epoch 1400 = 0.07510470194179018
Error on this batch = 0.43509373485870884
Error on this batch = 0.4351231623268474
Cost on val dataset after 1401 epochs is = 0.07510425884745377
learning rate =  0.134053620332232
Initial Cost on Val dataset for this epoch 1401 = 0.07510425884745377
Error on this batch = 0.4350890207911622
Error on this batch = 0.43511965449288653
Cost on val dataset after 1402 epochs is = 0.07510381614336172
learning rate =  0.1340217407542565
Initial Cost on Val dataset for this epoch 1402 = 0.07510381614336172
Error on this batch = 0.4350842940127573
Error on this batch = 0.4351161456

Error on this batch = 0.43500721936286907
Cost on val dataset after 1434 epochs is = 0.07508988586485443
learning rate =  0.13301732461969987
Initial Cost on Val dataset for this epoch 1434 = 0.07508988586485443
Error on this batch = 0.43492485684452165
Error on this batch = 0.4350037098767029
Cost on val dataset after 1435 epochs is = 0.07508945897077182
learning rate =  0.13298641910636172
Initial Cost on Val dataset for this epoch 1435 = 0.07508945897077182
Error on this batch = 0.4349195745400766
Error on this batch = 0.43500020109700577
Cost on val dataset after 1436 epochs is = 0.07508903264478034
learning rate =  0.132955542295615
Initial Cost on Val dataset for this epoch 1436 = 0.07508903264478034
Error on this batch = 0.43491427173315045
Error on this batch = 0.43499669304992894
Cost on val dataset after 1437 epochs is = 0.0750886068908828
learning rate =  0.13292469414084288
Initial Cost on Val dataset for this epoch 1437 = 0.0750886068908828
Error on this batch = 0.43490894

Error on this batch = 0.4348883967973311
Cost on val dataset after 1468 epochs is = 0.07507571108727928
learning rate =  0.13198236528891355
Initial Cost on Val dataset for this epoch 1468 = 0.07507571108727928
Error on this batch = 0.43473307777494974
Error on this batch = 0.4348849198461223
Cost on val dataset after 1469 epochs is = 0.07507530546457472
learning rate =  0.1319524101436975
Initial Cost on Val dataset for this epoch 1469 = 0.07507530546457472
Error on this batch = 0.43472704423798575
Error on this batch = 0.43488144401357287
Cost on val dataset after 1470 epochs is = 0.07507490053364005
learning rate =  0.13192248217484528
Initial Cost on Val dataset for this epoch 1470 = 0.07507490053364005
Error on this batch = 0.43472098806915266
Error on this batch = 0.43487796930801403
Cost on val dataset after 1471 epochs is = 0.07507449629917126
learning rate =  0.1318925813392393
Initial Cost on Val dataset for this epoch 1471 = 0.07507449629917126
Error on this batch = 0.434714

In [26]:
test_accuracy

[3.8461538461538463,
 14.046153846153848,
 68.27692307692308,
 86.01538461538462,
 80.07692307692308,
 77.04615384615384,
 80.87692307692308,
 79.93846153846154]

In [27]:
epochs

[52, 4437, 9803, 7909, 14918, 9888, 10596, 6803]

In [30]:
valid_accuracy


[3.128205128205128,
 14.000000000000002,
 69.94871794871796,
 86.56410256410257,
 80.35897435897435,
 77.64102564102564,
 81.8974358974359,
 80.82051282051282]

In [40]:
1.5/np.power(2039, (1/3))

0.118291130898163